In [4]:
# string.fromat

import yaml
import import_ipynb
import Read_pdf as rp

def display_template(data):
    topic = data.keys()
    print('--------------模板列表-----------------')
    for item in topic:
        print(item + ':')
        for sub_item in data[item]:
            print(' -' + sub_item)

def get_variable(text):#抓取{ 及 }的位置並取出裏頭的字串變成dic的key
    my_pos = []
    my_pos2 = []
    pos = 0
    pos2 = 0
    while True:
        pos = text.find('{', pos)
        pos2 = text.find('}', pos2)
        if pos2 == -1:
            break
        my_pos.append(pos)
        my_pos2.append(pos2)
        pos += 1
        pos2 +=1

    keys = []
    for start,end in zip(my_pos, my_pos2):
        keys.append(text[start+1:end])
    d = dict.fromkeys(keys)
    del my_pos
    del my_pos2
    del keys
    return d


def bid_value(bid, dic):
    if bid.currency == '人民幣':
        prefixCur = 'RMB'
        suffixCur = ''
    else:
        prefixCur =''
        suffixCur = '元'

    def name_format(var):
        return '「' + str(var) +'」'

    def price_format(var):
        return prefixCur + str(var) + suffixCur

    for key in dic:
        if key == '次別':
            dic[key] = bid.sn
        elif key == '決包廠商':
            dic[key] = name_format(bid.award_vnd)
        elif key == '追加減':
            dic[key] = bid.change
        elif key == '公司別':
            dic[key] = bid.comp
        elif key == '廠區':
            dic[key] = bid.id[1:3]
        elif key == '預算金額':
            dic[key] = price_format(bid.budget)
        elif key == '報價金額':
            dic[key] = price_format(bid.vnd[bid.awd_sn]['quo'])
        elif key == '議價金額':
            dic[key] = price_format(bid.vnd[bid.awd_sn]['finalPrice'])
        elif key == '差異':
            dic[key] = bid.dif_text(bid.awd_sn)
        elif key == '工程名稱':
            dic[key] = bid.name
        elif key == '及催報':
            a = ''
            for vnd in bid.vnd:
                if vnd['type'] == '後報' or vnd['type'] == '後加':
                    a == '及催報'
                    break
            dic[key] = a
        elif key == '報價家數':
                dic[key] = len(bid.vnd)
        else:
            dic[key] = input('請輸入「'+key +'」的值:')
    return dic

if __name__ == '__main__':
    file_yaml = 'template.yaml'
    with open(file_yaml, 'r', encoding = 'utf8') as stream:
        data = yaml.load(stream, Loader = yaml.FullLoader)
    
#     print('--------------模板可用樣式列表-------------')
#     display_template(data)        

    pdf_file = 'd:/N2.pdf'
    bidcase = rp.open(pdf_file)
    
    is_bargain = lambda n :'有議價' if n else '無議價'
    is_overbudget = lambda n :'(超預算)' if n else '(低預算)'
    a = ''
    if bidcase.type == '一般案':
        a = is_overbudget(bidcase.is_overBudget(bidcase.cheapest_sn))
    
    print('--------------模板樣式[{type}][{bargain}]-------------'\
        .format(type = bidcase.type, bargain = is_bargain(bidcase.is_bargain) + a))
            
    text = data[bidcase.type][is_bargain(bidcase.is_bargain) + a]
    
    print(text)
    
    dict_tags = get_variable(text)
    dict_tags = bid_value(bidcase, dict_tags)

    print('----------------套版後-------------------')
    print(text.format(**dict_tags))

--------------模板樣式[一般案][有議價(超預算)]-------------
1.本案為{公司別}{廠區}「{工程名稱}」，預算編列{預算金額}，增值稅率需求為{預算稅率}%。
2.開標{及催報}後共計{報價家數}家廠商報價，{議價狀況}{最低廠商}由{報價金額}降至{議價金額}{最低判斷}{回議判斷}，{差異}，廠商反應本案{差異原因}。
3.擬以{議價金額}交{最低廠商}承攬，{廠商發票稅率}。



請輸入「預算稅率」的值: x
請輸入「議價狀況」的值: 3
請輸入「最低廠商」的值: 11233
請輸入「最低判斷」的值: 213213
請輸入「回議判斷」的值: zz
請輸入「差異原因」的值: 221
請輸入「廠商發票稅率」的值: 11


----------------套版後-------------------
1.本案為2NJJ3「昆山CCL三廠配料室T-316至含浸配管」，預算編列RMB 89,262，增值稅率需求為x%。
2.開標後共計4家廠商報價，311233由RMB 131,462降至RMB 116,000213213zz，較預算+29.95%，廠商反應本案221。
3.擬以RMB 116,000交11233承攬，11。

